In [1]:
spark

# I've already created the SparkContext as sc here, so I can call it as sc

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
# Homework #6

# This actually works. It's the first thing I've ever gotten to work.

listRDD = sc.parallelize([1,2,3,4], 2)
seqOp = (lambda local_result, list_element: (local_result[0] + list_element, local_result[1] + 1))
combOp = (lambda some_local_result, another_local_result: (some_local_result[0] + another_local_result[0], some_local_result[1] + another_local_result[1]))

listRDD.aggregate((1,0), seqOp, combOp)

Out[10]: (13, 4)

In [3]:
listRDD.aggregate((2,0), seqOp, combOp)

Out[5]: (16, 4)

In [4]:
# Week 6 Question 3
# Reading a text file just requires that you flip around the slashes?  Thanks!
# The KEY to this entire thing is that you have to read whatever file you're working with into the DBFS. You can't just read from elsewhere.

ads_RDD = sc.textFile("/FileStore/tables/Applied_Data_Science.txt")
# This shows me the first 10 lines of text in the book
ads_RDD.take(10)

Out[6]: ['Applied Data Science',
 'Ian Langmore Daniel Krasner',
 '2',
 'Contents',
 'I Programming Prerequisites 1',
 '1 Unix 2',
 '1.1 History and Culture . . . . . . . . . . . . . . . . . . . . . . . 2',
 '1.2 The Shell . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3',
 '1.3 Streams . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5',
 '1.3.1 Standard streams . . . . . . . . . . . . . . . . . . . . . 6']

In [5]:
# Step 2: Inspect the number of partitions (workers) used to store the dataset
numPartitions = ads_RDD.getNumPartitions()
print("Number of partitions (workers) storing the dataset = %d"%numPartitions)


Number of partitions (workers) storing the dataset = 2

In [6]:
# Step 3: Split each line into a list of words separated by a space from the dataset
# Here I did it in several steps because I don't seem to be able to chain them.

# def remove_punc(x):
#   punc = '!"#$%^,:;<=>?@[\]^_~"'
#   for ch in punc:
#       x = ch.replace(ch, '')
#   return x

# no_punc = ads_RDD.map(remove_punc)
# This isn't right.
# ads_RDD.map(lambda w: w.translate(None, string.punctuation))

words = ads_RDD.flatMap(lambda w: w.split(' ')).map(lambda x: x.lower())
# words.take(10)

# Step 4: Just get the words I want.
# In the web example, they eliminate stopwords by defining a list and then keeping the words not in the list.

long_words = words.filter(lambda x: len(x) > 5)
# long_words.take(10)

# Step 5: Transform filtered words into list of (word, 1) tuples for WordCount

word1_tuples = long_words.map(lambda w: (w, 1)) # This maps the words into (word, 1) tuples
# word1_tuples.take(10)

# Step 6: AGGREGATE the (word, 1) tuples into (word, count) tuples

word_count_tuples = word1_tuples.reduceByKey(lambda x, y: x + y)
word_count_tuples.take(10)

Out[8]: [('applied', 2),
 ('science', 9),
 ('krasner', 1),
 ('contents', 9),
 ('programming', 8),
 ('streams', 12),
 ('philosophy', 6),
 ('version', 21),
 ('online', 2),
 ('concepts', 6)]

In [7]:
# this gives me the list in ascending order. Can I get it in descending order?
top5 = word_count_tuples.takeOrdered(5, key = lambda x: -x[1])

In [8]:
print('The most used words in the Applied Data Science textbook are:')
for s in top5:
  print('  '+s[0]+' occurred '+str(s[1])+' times')

The most used words in the Applied Data Science textbook are:
 python occurred 88 times
 chapter occurred 76 times
 regression occurred 65 times
 number occurred 63 times
 function occurred 57 times

In [9]:
# Homework 7, Question 4 about CombineByKey

data = sc.parallelize( [('Maryland', 2), ('Virginia', 5), ('Delaware', 1), ('Maryland', 3), ('Delaware', 1)], 2 )

step1 = data.groupByKey().mapValues(lambda x: list(x)).collect()

sumCount = data.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))


sumCount.take(10)
# averageByKey = sumCount.map(lambda (label, (value_sum, count)): (label, value_sum / count))

# # print(averageByKey.collectAsMap())

Out[11]: [('Maryland', (5, 2)), ('Delaware', (2, 2)), ('Virginia', (5, 1))]

In [10]:
print(step1)

[('Maryland', [2, 3]), ('Delaware', [1, 1]), ('Virginia', [5])]

In [11]:
# Homework #6 Question 2

# Write a Spark program that reads your browser history file, then displays the top 5 websites you visited in the last week.

web_history = sc.textFile("/FileStore/tables/web_history.json")


In [12]:
web_history.take(100)

Out[14]: ['[{',
 '\t"id": 72464,',
 '\t"title": "Add & open Chrome apps - Chrome Web Store Help",',
 '\t"url": "https://support.google.com/chrome_webstore/answer/3060053?hl=en",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:38:24 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:38:24.974Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72463,',
 '\t"title": "how do i open something i downloaded from the chrome web store - Google Search",',
 '\t"url": "https://www.google.com/search?q=how+do+i+open+something+i+downloaded+from+the+chrome+web+store&oq=how+do+i+open+something+i+downloaded+from+the+chrome+web+sto&aqs=chrome.1.69i57j33.15382j1j4&sourceid=chrome&ie=UTF-8",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:38:18 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:38:18.902Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 2',
 '},{',
 '\t"id": 72462,',
 '\t"title": "Export History/Bookmarks to JSON/CSV*/XLS* - Chrome Web Store",',
 '\t"url": "https://chrome.google.com/webstore/detail/export-historybookmarks-t/dcoegfodcnjofhjfbhegcgjgapeichlf?hl=en-US&authuser=1",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:37:52 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:37:52.690Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72455,',
 '\t"title": "json-xls.com - json/xml to excel/CSV/html online converter and viewer",',
 '\t"url": "http://www.json-xls.com/",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:37:40 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:37:40.245Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 4',
 '},{',
 '\t"id": 72461,',
 '\t"title": "json-xls.com - Statistics",',
 '\t"url": "http://www.json-xls.com/statistics",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:37:06 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:37:06.834Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72460,',
 '\t"title": "json-xls.com",',
 '\t"url": "http://www.json-xls.com/api",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:37:06 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:37:06.052Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72459,',
 '\t"title": "json-xls.com - JSON Viewer",',
 '\t"url": "http://www.json-xls.com/json-viewer",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:37:01 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:37:01.652Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72458,',
 '\t"title": "Can Chrome browser history be exported to an HTML file? - Super User",',
 '\t"url": "https://superuser.com/questions/602252/can-chrome-browser-history-be-exported-to-an-html-file",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:34:01 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:34:01.579Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 1',
 '},{',
 '\t"id": 72457,',
 '\t"title": "how to use Export History/Bookmarks to JSON/CSV*/XLS* - Google Search",',
 '\t"url": "https://www.google.com/search?ei=FLr0XoDuH_-HytMPo9O_2AU&q=how+to+use+Export+History%2FBookmarks+to+JSON%2FCSV*%2FXLS*&oq=how+to+use+Export+History%2FBookmarks+to+JSON%2FCSV*%2FXLS*&gs_lcp=CgZwc3ktYWIQAzIFCAAQzQI6BQgAEJECOgUIABCxAzoECAAQQzoFCAAQgwE6AggAUNcNWP4ZYN4eaABwAHgAgAF3iAHoBpIBBDEwLjKYAQCgAQGgAQKqAQdnd3Mtd2l6&sclient=psy-ab&ved=0ahUKEwiAyO7im53qAhX_g3IEHaPpD1sQ4dUDCAw&uact=5",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:33:54 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:33:54.128Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 2',
 '},{',
 '\t"id": 72448,',
 '\t"title": "export history/bookmarks to json/csv*/xls* - Google Search",',
 '\t"url": "https://www.google.com/search?q=export+history%2Fbookmarks+to+json%2Fcsv*%2Fxls*&oq=export+histor%2Fbookmarks+to+json&aqs=chrome.1.69i57j0l2.7315j1j4&sourceid=chrome&ie=UTF-8",',
 '\t"lastVisitTimeLocal": "6/25/2020, 2:33:48 PM",',
 '\t"lastVisitTimeUTC": "2020-06-25T18:33:48.506Z",',
 '\t"typedCount": 0,',
 '\t"visitCount": 4',
 '},{',
 '\t"id": 72453,',
 '\t"title": "Export History/Bookmarks to JSON/CSV*/XLS* - Chrome Web Store",',
 '\t"url": "https://chrome.google.com/webstore/detail/

In [13]:
# By making this multiLine = True I was able to read the JSON file

path = "/FileStore/tables/web_history.json"
testDF = spark.read.json(path, multiLine = True)
testDF.show()

+-----+--------------------+--------------------+--------------------+----------+--------------------+----------+
 id| lastVisitTimeLocal| lastVisitTimeUTC| title|typedCount| url|visitCount|
+-----+--------------------+--------------------+--------------------+----------+--------------------+----------+
72464|6/25/2020, 2:38:2...|2020-06-25T18:38:...|Add & open Chrome...| 0|https://support.g...| 1|
72463|6/25/2020, 2:38:1...|2020-06-25T18:38:...|how do i open som...| 0|https://www.googl...| 2|
72462|6/25/2020, 2:37:5...|2020-06-25T18:37:...|Export History/Bo...| 0|https://chrome.go...| 1|
72455|6/25/2020, 2:37:4...|2020-06-25T18:37:...|json-xls.com - js...| 0|http://www.json-x...| 4|
72461|6/25/2020, 2:37:0...|2020-06-25T18:37:...|json-xls.com - St...| 0|http://www.json-x...| 1|
72460|6/25/2020, 2:37:0...|2020-06-25T18:37:...| json-xls.com| 0|http://www.json-x...| 1|
72459|6/25/2020, 2:37:0...|2020-06-25T18:37:...|json-xls.com - JS...| 0|http://www.json-x...| 1|
72458|6/25/2020, 2:34:0...|2020-06-25T18:34:...|Can Chrome browse...| 0|https://superuser...| 1|
72457|6/25/2020, 2:33:5...|2020-06-25T18:33:...|how to use Export...| 0|https://www.googl...| 2|
72448|6/25/2020, 2:33:4...|2020-06-25T18:33:...|export history/bo...| 0|https://www.googl...| 4|
72453|6/25/2020, 2:33:3...|2020-06-25T18:33:...|Export History/Bo...| 0|https://chrome.go...| 2|
72456|6/25/2020, 2:33:2...|2020-06-25T18:33:...|Export History - ...| 0|https://chrome.go...| 1|
72449|6/25/2020, 2:32:1...|2020-06-25T18:32:...|Download Export H...| 0|https://www.crx4c...| 3|
72452|6/25/2020, 2:30:4...|2020-06-25T18:30:...|Export History/Bo...| 0|https://chrome.go...| 1|
72450|6/25/2020, 2:30:1...|2020-06-25T18:30:...|Export History/Bo...| 0|https://www.crx4c...| 1|
72073|6/25/2020, 9:36:2...|2020-06-25T13:36:...| Citrix Workspace| 0|https://ctxextron...| 4|
72070|6/25/2020, 9:30:5...|2020-06-25T13:30:...| Citrix Workspace| 0|https://ctxextron...| 16|
19835|6/25/2020, 8:58:0...|2020-06-25T12:58:...|Inbox (18) - mich...| 16|https://mail.goog...| 79|
72447|6/25/2020, 8:57:2...|2020-06-25T12:57:...|Launch Meeting - ...| 0|https://extron.zo...| 1|
72071|6/25/2020, 8:57:1...|2020-06-25T12:57:...| Citrix Workspace| 0|https://ctxextron...| 5|
+-----+--------------------+--------------------+--------------------+----------+--------------------+----------+
only showing top 20 rows

In [14]:
testDF.orderBy('visitCount', ascending = False).select('url', 'visitCount').show(5)

+--------------------+----------+
 url|visitCount|
+--------------------+----------+
https://thechive....| 2591|
https://www.washi...| 991|
https://mail.goog...| 459|
https://my3.my.um...| 309|
https://blackboar...| 271|
+--------------------+----------+
only showing top 5 rows

In [15]:
# OK.  This seems to resemble the schema that my JSON file has.
# Because it's in JSON format, it automatically creates a dataframe.

# Now I want to sort by visitCount in descending order

testDF.select('lastVisitTimeLocal').show()

+--------------------+
 lastVisitTimeLocal|
+--------------------+
6/25/2020, 2:38:2...|
6/25/2020, 2:38:1...|
6/25/2020, 2:37:5...|
6/25/2020, 2:37:4...|
6/25/2020, 2:37:0...|
6/25/2020, 2:37:0...|
6/25/2020, 2:37:0...|
6/25/2020, 2:34:0...|
6/25/2020, 2:33:5...|
6/25/2020, 2:33:4...|
6/25/2020, 2:33:3...|
6/25/2020, 2:33:2...|
6/25/2020, 2:32:1...|
6/25/2020, 2:30:4...|
6/25/2020, 2:30:1...|
6/25/2020, 9:36:2...|
6/25/2020, 9:30:5...|
6/25/2020, 8:58:0...|
6/25/2020, 8:57:2...|
6/25/2020, 8:57:1...|
+--------------------+
only showing top 20 rows

In [16]:
last_week_df = testDF.filter(testDF['lastVisitTimeLocal'] > '6/18/2020, 0:00:01').orderBy('visitCount', ascending = False).select('url', 'visitCount').show(5)

+--------------------+----------+
 url|visitCount|
+--------------------+----------+
https://thechive....| 2591|
https://www.washi...| 991|
https://mail.goog...| 459|
https://my3.my.um...| 309|
https://blackboar...| 271|
+--------------------+----------+
only showing top 5 rows

+----------+
visitCount|
+----------+
 1|
 2|
 1|
 4|
 1|
 1|
 1|
 1|
 2|
 4|
 2|
 1|
 3|
 1|
 1|
 4|
 16|
 79|
 1|
 5|
+----------+
only showing top 20 rows

In [19]:
# Here I explored putting it in a df, but it turns out I didn't need to do this.
df = word_count_tuples.toDF(['Word', 'Count'])

In [20]:
# More stuff I didn't need to do after all

from pyspark.sql.functions import desc

finished_df = df.sort(desc('Count'))
finished_df.show()

+-----------+-----+
 Word|Count|
+-----------+-----+
 python| 88|
 chapter| 76|
 regression| 65|
 number| 63|
 function| 57|
 figure| 53|
 linear| 53|
 suppose| 50|
 problem| 50|
 training| 49|
 logistic| 41|
 example,| 40|
 memory| 38|
 variable| 38|
 exercise| 36|
 regular| 36|
 example| 36|
 likelihood| 36|
coefficient| 35|
 solution| 35|
+-----------+-----+
only showing top 20 rows

In [21]:
lines = sc.textFile("/FileStore/tables/Applied_Data_Science.txt")

numPartitions = lines.getNumPartitions() # get the number of partitions

words = lines.flatMap(lambda x: x.split(' ')) # split each line into a list of words
stopWords = ['','a','*','and','is','of','the','a', '.', '='] # define the list of stop words
filteredWords = words.filter(lambda x: x.lower() not in stopWords) # filter the words

filteredWords.cache() # cache filtered dataset into memory across the cluster worker nodes I DID NOT DO THIS

word1Tuples = filteredWords.map(lambda x: (x, 1)) # map the words into (word,1) tuples

wordCountTuples = word1Tuples.reduceByKey(lambda x, y: x + y) # aggregate counts for each word

sortedWordCountTuples = wordCountTuples.takeOrdered(10, key = lambda x: -x[1])
for tuple in sortedWordCountTuples: # display the top 10 (word,count) tuples by count 
  print(str(tuple))

('to', 759)
('that', 442)
('in', 403)
('we', 280)
('be', 264)
('for', 257)
('as', 256)
('with', 253)
('can', 231)
('are', 230)

In [22]:
word1Tuples.take(10)

Out[22]: [('Applied', 1),
 ('Data', 1),
 ('Science', 1),
 ('Ian', 1),
 ('Langmore', 1),
 ('Daniel', 1),
 ('Krasner', 1),
 ('2', 1),
 ('Contents', 1),
 ('I', 1)]

In [23]:
# Trying Question 2


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191286> in <module> 
 1 from pyspark . sql import Row # import the pyspark sql Row class 
 ----> 2 word_count_rows = wordCountTuples . map ( lambda p : Row ( word = p [ 0 ] , count = int ( p [ 1 ] ) ) ) # tuples -> Rows 
 3 word_count_rows . toDF ( ) . createOrReplaceTempView ( "word_count" ) 

 NameError : name 'wordCountTuples' is not defined

In [26]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-1626975377870626> in <module> 
 ----> 1 from bs4 import BeautifulSoup
 2 from urllib . request import urlopen
 3 import requests

 ModuleNotFoundError : No module named 'bs4'

In [27]:
umbc_html = requests.get('https://www.umbc.edu')
umbc_text = umbc_html.text

print('Length of html text is '+str(len(umbc_text))+' characters.')
print(umbc_text)

url = 'https://www.umbc.edu'
umbc_html = urlopen(url)

soup = BeautifulSoup(umbc_html)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191276> in <module> 
 ----> 1 umbc_html = requests . get ( 'https://www.umbc.edu' ) 
 2 umbc_text = umbc_html . text
 3 
 4 print ( 'Length of html text is ' + str ( len ( umbc_text ) ) + ' characters.' ) 
 5 print ( umbc_text ) 

 NameError : name 'requests' is not defined

In [28]:
images = soup.findAll('img')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191287> in <module> 
 ----> 1 images = soup . findAll ( 'img' ) 

 NameError : name 'soup' is not defined

In [29]:
images

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191294> in <module> 
 ----> 1 images

 NameError : name 'images' is not defined

In [30]:
print('There are '+str(len(images))+' images at the '+url+' site.')
print('\nThe images are:\n')
for image in images:
    print('  '+image['src'])

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191292> in <module> 
 ----> 1 print ( 'There are ' + str ( len ( images ) ) + ' images at the ' + url + ' site.' ) 
 2 print ( '\nThe images are:\n' ) 
 3 for image in images : 
 4 print ( ' ' + image [ 'src' ] ) 

 NameError : name 'images' is not defined

In [31]:
# Here I'm trying the same thing, but with a text file

In [32]:
# Here I read the Text version of the html into a text file
umbc = sc.textFile('/FileStore/tables/umbc.txt')

In [33]:
umbc1 = umbc.filter(lambda x:'img/' in x)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1204972937191284> in <module> 
 ----> 1 for line in umbc1 : 
 2 line . filter ( lambda x : 'img/' in x ) 
 3 print ( line ) 

 TypeError : 'PipelinedRDD' object is not iterable

In [35]:
umbc2.count()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191290> in <module> 
 ----> 1 umbc2 . count ( ) 

 NameError : name 'umbc2' is not defined

In [36]:
umbc2.take(1)
# for line in umbc1.take(2): print(line)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1204972937191291> in <module> 
 ----> 1 umbc2 . take ( 1 ) 
 2 # for line in umbc1.take(2): print(line) 

 NameError : name 'umbc2' is not defined

In [37]:
%sh curl -O 'https://raw.githubusercontent.com/mschlitzer/2019_Footy_Player_Dictionary/master/footy_stats.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 49494 100 49494 0 0 154k 0 --:--:-- --:--:-- --:--:-- 154k

In [38]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/footy_stats.csv,footy_stats.csv,49494
file:/databricks/driver/derby.log,derby.log,717
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/logs/,logs/,4096


In [39]:
path = 'file:/databricks/driver/footy_stats.csv'

df = sqlContext.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', 'true')\
.load(path)

In [40]:
display(df)

Player,Team,Position
Luke Brown,Adelaide,Defender
Tom Doedee,Adelaide,Defender
Kyle Hartigan,Adelaide,Defender
Jake Kelly,Adelaide,Defender
Rory Laird,Adelaide,Defender
David Mackay,Adelaide,Defender
Daniel Talia,Adelaide,Defender
Wayne Milera,Adelaide,Defender
Andy Otten,Adelaide,Defender
Kyle Cheney,Adelaide,Defender


Out[22]: Column<b'Team'>

In [42]:


numeric_val = sc.parallelize([1,2,3,4])
numeric_val.map(lambda x: x**3).collect()

Out[37]: [1, 8, 27, 64]

In [43]:
# Class for today, June 22 2020

In [44]:
basicRDD = sc.parallelize([1,2,3,3])

In [45]:
basicRDD.map(lambda x: x+1)

Out[40]: PythonRDD[963] at RDD at PythonRDD.scala:59

In [46]:
nums = sc.parallelize([1,2,3,4])
Squared = nums.map(lambda x: x*x).collect()

In [47]:
for num in Squared:
  print("%i" %(num))

1
4
9
16

In [48]:
# Homework 9 in Spark SQL

# Create a dataframe in spark sql and do stuff to it

# I'm just going to follow the example from the Databricks tutorial until it breaks.

from pyspark.sql import *

In [49]:
# Create my sample data - Students and GPA.  Later I will create a new column for a letter grade.

# Department data
department1 = Row(id = '100', name = 'Computer Science')
department2 = Row(id = '200', name = 'Data Science')
department3 = Row(id = '300', name = 'Political Science')
department4 = Row(id = '400', name = 'History')

# Student data
# Pay attention here.  I create the row as Student and then assign individual students to the Student object
Student = Row('StudentID', 'LastName', 'FirstName', 'PhoneNumber', 'GPA')
student1 = Student(541, 'Washington', 'George', '703-703-1234', 4.0)
student2 = Student(542, 'Adams', 'John', '703-703-1235', 3.75)
student3 = Student(543, 'Jefferson', 'Thomas', '703-703-1236', 3.5)
student4 = Student(544, 'Franklin', 'Ben', '703-703-1237', 3.25)
student5 = Student(545, 'Monroe', 'James', '703-703-1238', 3.0)
student6 = Student(546, 'Henry', 'Patrick', '703-703-1239', 2.75)
student7 = Student(547, 'Mason', 'George', '703-703-1240', 2.5)
student8 = Student(548, 'Madison', 'James', '703-703-1241', 2.25)
student9 = Student(549, 'Burr', 'Aaron', '703-703-1242', 2.0)
student10 = Student(550, 'Hamilton', 'Alexander', '703-703-1243', 1.75)

# Create the StudentsWithDepartments instances from Departments and Students

departmentWithStudents1 = Row(department = department1, students = [student1, student2, student3])
departmentWithStudents2 = Row(department = department2, students = [student4, student5, student6])
departmentWithStudents3 = Row(department = department3, students = [student7, student8])
departmentWithStudents4 = Row(department = department4, students = [student9, student10])

print(department1)
print(student2)
print(departmentWithStudents1.students[0].FirstName)


Row(id='100', name='Computer Science')
Row(StudentID=542, LastName='Adams', FirstName='John', PhoneNumber='703-703-1235', GPA=3.75)
George

In [50]:
departmentsWithStudentsSeq1 = [departmentWithStudents1, departmentWithStudents2]
df1 = spark.createDataFrame(departmentsWithStudentsSeq1)

departmentsWithStudentsSeq2 = [departmentWithStudents2, departmentWithStudents4]
df2 = spark.createDataFrame(departmentsWithStudentsSeq2)

In [51]:
# Work with DataFrames

unionDF = df1.union(df2)
display(unionDF)

department,students
"List(100, Computer Science)","List(List(541, Washington, George, 703-703-1234, 4.0), List(542, Adams, John, 703-703-1235, 3.75), List(543, Jefferson, Thomas, 703-703-1236, 3.5))"
"List(200, Data Science)","List(List(544, Franklin, Ben, 703-703-1237, 3.25), List(545, Monroe, James, 703-703-1238, 3.0), List(546, Henry, Patrick, 703-703-1239, 2.75))"
"List(200, Data Science)","List(List(544, Franklin, Ben, 703-703-1237, 3.25), List(545, Monroe, James, 703-703-1238, 3.0), List(546, Henry, Patrick, 703-703-1239, 2.75))"
"List(400, History)","List(List(549, Burr, Aaron, 703-703-1242, 2.0), List(550, Hamilton, Alexander, 703-703-1243, 1.75))"


In [52]:
# Write the unioned DataFrame to a Parquet File

dbutils.fs.rm("tmp/databricks-df-example.parquet", True)
unionDF.write.parquet("tmp/databricks-df-example.parquet")

# This goddamned thing.  It doesn't like spaces.  Great fucking error messaging.

In [53]:
from  pyspark.sql.functions import explode

explodeDF = unionDF.select(explode("students").alias("s"))
flattenDF =  explodeDF.selectExpr("s.StudentID", "s.LastName", "s.FirstName", "s.GPA")

flattenDF.show()

+---------+----------+---------+----+
StudentID| LastName|FirstName| GPA|
+---------+----------+---------+----+
 541|Washington| George| 4.0|
 542| Adams| John|3.75|
 543| Jefferson| Thomas| 3.5|
 544| Franklin| Ben|3.25|
 545| Monroe| James| 3.0|
 546| Henry| Patrick|2.75|
 544| Franklin| Ben|3.25|
 545| Monroe| James| 3.0|
 546| Henry| Patrick|2.75|
 549| Burr| Aaron| 2.0|
 550| Hamilton|Alexander|1.75|
+---------+----------+---------+----+

In [54]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

def assign_letter_grade(score): 
    if score >= 3.33: return "A"
    elif score >= 2.67: return "B"
    elif score >= 1.67: return "C"
    elif score >= 0.67: return "D"
    else : return "F"

udf_grade = F.udf(assign_letter_grade, StringType())

grade_letter_columnDF = flattenDF.withColumn("LetterGrade", udf_grade(flattenDF.GPA))
# newDF = flattenDF.withColumn('LetterGrade', F.when((F.col("GPA") >= 3.33), 'A'))
# F.when((F.col("GPA") >= 3.33), 'A'))



In [55]:
# newDF.show()
grade_letter_columnDF.show()

+---------+----------+---------+----+-----------+
StudentID| LastName|FirstName| GPA|LetterGrade|
+---------+----------+---------+----+-----------+
 541|Washington| George| 4.0| A|
 542| Adams| John|3.75| A|
 543| Jefferson| Thomas| 3.5| A|
 544| Franklin| Ben|3.25| B|
 545| Monroe| James| 3.0| B|
 546| Henry| Patrick|2.75| B|
 544| Franklin| Ben|3.25| B|
 545| Monroe| James| 3.0| B|
 546| Henry| Patrick|2.75| B|
 549| Burr| Aaron| 2.0| C|
 550| Hamilton|Alexander|1.75| C|
+---------+----------+---------+----+-----------+

In [56]:
grade_letter_columnDF.write.parquet("tmp/databricks-df-grade_letter.parquet")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2056.parquet.
: org.apache.spark.sql.AnalysisException: path dbfs:/tmp/databricks-df-grade_letter.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:150)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:138)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:191)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:187)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:138)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:117)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1$$anonfun$apply$1.apply(SQLExecution.scala:112)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:232)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:98)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:835)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:74)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:184)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:600)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-1619531716250247> in <module> 
 ----> 1 grade_letter_columnDF . write . parquet ( "tmp/databricks-df-grade_letter.parquet" ) 

 /databricks/spark/python/p

In [57]:
# At the very least this worked.  I'm not sure what it does, but it works.

from os.path import expanduser, join, abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

warehouse_location = abspath('spark-warehouse')
spark = SparkSession \
.builder \
.appName('Python Spark SQL Hive integration example')\
.config('spark.sql.warehouse.dir', warehouse_location)\
.enableHiveSupport()\
.getOrCreate()

In [58]:
spark.sql('CREATE TABLE IF NOT EXISTS Data1 (StudentID INT, StudentName STRING, PhoneNumber STRING, GPA DECIMAL(3,2))')
# spark.sql("LOAD DATA '/FileStore/tables/homework9.csv' INTO TABLE Data1")

# so the problem here is that I don't know where my spark-warehouse is to put a file

# /FileStore/tables/homework9.csv


Out[53]: DataFrame[]

In [59]:
# Now we're going to Explode the students column.  What a fun word!
from pyspark.sql.functions import explode

explodeDF = unionDF.select(explode('students').alias('s'))
flattenDF = explodeDF.selectExpr('s.StudentID','s.LastName', 's.FirstName', 's.PhoneNumber', 's.GPA')

flattenDF.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o917.selectExpr.
: org.apache.spark.sql.AnalysisException: No such struct field StudentID in Student ID, LastName, FirstName, PhoneNumber, GPA; line 1 pos 0
	at org.apache.spark.sql.catalyst.expressions.ExtractValue$.findField(complexTypeExtractors.scala:85)
	at org.apache.spark.sql.catalyst.expressions.ExtractValue$.apply(complexTypeExtractors.scala:53)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq$$anonfun$7.apply(package.scala:232)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq$$anonfun$7.apply(package.scala:231)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:231)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:919)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:921)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:53)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:918)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1005)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1005)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.colle

In [60]:
# Now we're going to Explode the students column.  What a fun word!
from pyspark.sql.functions import explode

explodeDF = unionDF.select(explode('students').alias('s'))
flattenDF = explodeDF.selectExpr('s.StudentID','s.LastName', 's.FirstName', 's.PhoneNumber', 's.GPA')

flattenDF.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o917.selectExpr.
: org.apache.spark.sql.AnalysisException: No such struct field StudentID in Student ID, LastName, FirstName, PhoneNumber, GPA; line 1 pos 0
	at org.apache.spark.sql.catalyst.expressions.ExtractValue$.findField(complexTypeExtractors.scala:85)
	at org.apache.spark.sql.catalyst.expressions.ExtractValue$.apply(complexTypeExtractors.scala:53)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq$$anonfun$7.apply(package.scala:232)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq$$anonfun$7.apply(package.scala:231)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:231)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:919)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:921)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:53)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:918)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:927)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1005)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1005)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.colle

In [61]:
%sh
pip install koalas

Collecting koalas
 Downloading https://files.pythonhosted.org/packages/50/b6/86413b835482398002b0609c337113e61998c00e3b0183b8adfaba9c56ee/koalas-1.0.1-py3-none-any.whl (691kB)
Requirement already satisfied: matplotlib>=3.0.0 in /databricks/python3/lib/python3.7/site-packages (from koalas) (3.0.3)
Requirement already satisfied: pyarrow>=0.10 in /databricks/python3/lib/python3.7/site-packages (from koalas) (0.13.0)
Requirement already satisfied: pandas>=0.23.2 in /databricks/python3/lib/python3.7/site-packages (from koalas) (0.24.2)
Requirement already satisfied: numpy>=1.14 in /databricks/python3/lib/python3.7/site-packages (from koalas) (1.16.2)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (2.8.0)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (1.1.0)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (2.4.6)
Requirement already satisfied: six>=1.0.0 in /databricks/python3/lib/python3.7/site-packages (from pyarrow>=0.10->koalas) (1.12.0)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.23.2->koalas) (2018.9)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from kiwisolver>=1.0.1->matplotlib>=3.0.0->koalas) (40.8.0)
Installing collected packages: koalas
Successfully installed koalas-1.0.1
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [62]:
import databricks.koalas as ks

In [63]:
data = ks.read_csv('/FileStore/tables/homework9.csv')


In [64]:
def assign_letter_grade(score): 
    if score >= 3.33: return "A"
    elif score >= 2.67: return "B"
    elif score >= 1.67: return "C"
    elif score >= 0.67: return "D"
    else : return "F"

grades = []
for i in range(len(data)):
  answer = assign_letter_grade(data['GPA'].values[i])
  grades.append(answer)
gradesdf = ks.DataFrame({'LetterGrades': grades})
mergeddf = data.merge(gradesdf, left_index = True, right_index = True)
mergeddf

,Student ID,LastName,FirstName,PhoneNumber,GPA,LetterGrades
0,541,Washington,George,703-703-1234,4.00,A
1,542,Adams,John,703-703-1235,3.75,A
2,543,Jefferson,Thomas,703-703-1236,3.50,A
3,544,Franklin,Ben,703-703-1237,3.25,B
4,545,Monroe,James,703-703-1238,3.00,B
5,546,Henry,Patrick,703-703-1239,2.75,B
6,547,Mason,George,703-703-1240,2.50,C
7,548,Madison,James,703-703-1241,2.25,C
8,549,Burr,Aaron,703-703-1242,2.00,C
9,550,Hamilton,Alexander,703-703-1243,1.75,C


In [65]:
mergeddf

,Student ID,LastName,FirstName,PhoneNumber,GPA,LetterGrades
0,541,Washington,George,703-703-1234,4.00,A
1,542,Adams,John,703-703-1235,3.75,A
2,543,Jefferson,Thomas,703-703-1236,3.50,A
3,544,Franklin,Ben,703-703-1237,3.25,B
4,545,Monroe,James,703-703-1238,3.00,B
5,546,Henry,Patrick,703-703-1239,2.75,B
6,547,Mason,George,703-703-1240,2.50,C
7,548,Madison,James,703-703-1241,2.25,C
8,549,Burr,Aaron,703-703-1242,2.00,C
9,550,Hamilton,Alexander,703-703-1243,1.75,C


,Student ID,LastName,FirstName,PhoneNumber,GPA
LetterGrades,,,,,
B,3,3,3,3,3
C,4,4,4,4,4
A,3,3,3,3,3


In [67]:
# Homework 11, Machine Learning

In [68]:
flight_distances = ks.read_csv('/FileStore/tables/flight_distances.csv')

In [69]:
flight_distances = flight_distances.set_index('_c0')

In [70]:
flight_distances

,IAD,ORD,BOS,ATL,LAX,DEN,SFO,LGA,MIA,SEA
_c0,,,,,,,,,,
IAD,0,588,413,534,2288,1452,2419,229,921,2306
ORD,588,0,867,606,1744,888,1846,733,1197,1721
BOS,413,867,0,946,2611,1754,2704,184,1258,2496
ATL,534,606,946,0,1947,1199,2139,762,594,2182
LAX,2288,1744,2611,1947,0,862,337,2469,2342,954
DEN,1452,888,1754,1199,862,0,967,1620,1709,1024
SFO,2419,1846,2704,2139,337,967,0,2579,2585,679
LGA,229,733,184,762,2469,1620,2579,0,1096,2413
MIA,921,1197,1258,594,2342,1709,2585,1096,0,2724


In [71]:
flight = flight_distances.loc['ORD', 'SFO']

flight

Out[7]: 1846

In [72]:
import random
airports = list(flight_distances.columns)


flight_departures = []
flight_arrivals = []

# OK, I don't have to worry about these being in list formats inside of the outer list

n = 0
while n < 100:
  departure = random.choices(airports)
  flight_departures.append(departure)
  n += 1

for f in flight_departures:
  arrival = random.choices(airports)
  if f == arrival:
    arrival = random.choices(airports)
  flight_arrivals.append(arrival)
  
df_d = ks.DataFrame(flight_departures)
df_a = ks.DataFrame(flight_arrivals)

df_temp = df_d.merge(df_a, left_index = True, right_index = True)

In [73]:
# so here I can just use that same logic to compute time, right?

distances = []
for i, r in df_temp.iterrows():
  mph = random.randrange(460, 500)
  mpm = mph / 60
  distance = flight_distances.loc[r['0_x'], r['0_y']]
  duration = distance * mpm
  distances.append([r['0_x'], r['0_y'], distance, duration])
  



In [74]:
# So, what I have here is a list of originating airports, destination airports, Distances in miles, and the number of minutes that the flight
# should take.  I have randomized the speed of the plane between the known operating speeds of the jets, between 460 and 500 mph.

# That introduces variability.  Now, no two distances are the same, so I am going to try and solve this with my X being column 2 and my y being column 3

df_dist = ks.DataFrame(distances)


In [75]:
df_dist.columns = ['Departure', 'Arrival', 'Mileage', 'Duration (min)']

spark_df = df_dist.to_spark()

In [76]:
spark_df.show(5)

+---------+-------+-------+--------------+
Departure|Arrival|Mileage|Duration (min)|
+---------+-------+-------+--------------+
 LGA| LAX| 2469| 19752.0|
 DEN| LAX| 862| 6809.8|
 LAX| IAD| 2288| 18876.0|
 MIA| DEN| 1709| 13928.35|
 SFO| BOS| 2704| 21496.8|
+---------+-------+-------+--------------+
only showing top 5 rows

In [77]:
from pyspark.ml.feature import VectorAssembler

stages = []

numericCols = ['Mileage']
assemblerInputs = numericCols
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = "features")


In [78]:
spark_df_2 = assembler.transform(spark_df)
spark_df_2.show()

+---------+-------+-------+------------------+--------+
Departure|Arrival|Mileage| Duration (min)|features|
+---------+-------+-------+------------------+--------+
 BOS| MIA| 1258|10084.966666666667|[1258.0]|
 BOS| MIA| 1258|10210.766666666666|[1258.0]|
 SFO| IAD| 2419|19674.533333333333|[2419.0]|
 ATL| SFO| 2139| 17147.65|[2139.0]|
 SEA| ATL| 2182| 16910.5|[2182.0]|
 LAX| BOS| 2611|20409.316666666666|[2611.0]|
 SEA| DEN| 1024| 8465.066666666668|[1024.0]|
 MIA| ORD| 1197| 9735.599999999999|[1197.0]|
 SFO| SEA| 679| 5239.616666666667| [679.0]|
 ATL| ORD| 606| 4959.1| [606.0]|
 BOS| LGA| 184|1465.8666666666668| [184.0]|
 MIA| IAD| 921| 7245.2| [921.0]|
 ORD| LAX| 1744|13428.800000000001|[1744.0]|
 MIA| SEA| 2724| 20884.0|[2724.0]|
 ORD| SFO| 1846|14798.766666666668|[1846.0]|
 ATL| MIA| 594| 4910.400000000001| [594.0]|
 BOS| IAD| 413|3214.5166666666664| [413.0]|
 DEN| SEA| 1024| 7953.066666666667|[1024.0]|
 IAD| ATL| 534| 4378.799999999999| [534.0]|
 DEN| LAX| 862| 7097.133333333332| [862.0]|
+---------+-------+-------+------------------+--------+
only showing top 20 rows

In [79]:
train, test = spark_df_2.randomSplit([0.9, 0.1])

In [80]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol = 'features', labelCol = 'Duration (min)')

In [81]:
model = algo.fit(train)

In [82]:
evaluation_summary = model.evaluate(test)

In [83]:
print(evaluation_summary.meanAbsoluteError)
print(evaluation_summary.rootMeanSquaredError)
print(evaluation_summary.r2)

282.1609818435481
315.2201029257655
0.9961624471013434

In [84]:
predictions = model.transform(test)

In [85]:
predictions.show()

+---------+-------+-------+------------------+--------+------------------+
Departure|Arrival|Mileage| Duration (min)|features| prediction|
+---------+-------+-------+------------------+--------+------------------+
 LAX| BOS| 2611|20409.316666666666|[2611.0]|20821.559145038154|
 SEA| ATL| 2182| 16910.5|[2182.0]| 17401.77011638372|
 SEA| DEN| 1024| 8465.066666666668|[1024.0]| 8170.731199875941|
 SFO| DEN| 967| 7897.166666666666| [967.0]| 7716.353636628149|
 SFO| SEA| 679| 5624.383333333333| [679.0]| 5420.551211797198|
 ORD| MIA| 1197| 9915.15|[1197.0]| 9549.806962013976|
 SFO| IAD| 2419| 18747.25|[2419.0]|19291.024195150854|
 ORD| SEA| 1721|13968.783333333335|[1721.0]|13726.891929414733|
 ATL| BOS| 946| 7315.733333333334| [946.0]| 7548.951376484225|
 ATL| LAX| 1947|15770.699999999999|[1947.0]|15528.459110011243|
 ORD| ATL| 606| 4706.6| [606.0]| 4838.629069392131|
 SEA| ORD| 1721| 13681.95|[1721.0]|13726.891929414733|
+---------+-------+-------+------------------+--------+------------------+

In [86]:
display(predictions)

Departure,Arrival,Mileage,Duration (min),features,prediction
LAX,BOS,2611,20409.316666666666,"List(1, 1, List(), List(2611.0))",20821.559145038154
SEA,ATL,2182,16910.5,"List(1, 1, List(), List(2182.0))",17401.77011638372
SEA,DEN,1024,8465.066666666668,"List(1, 1, List(), List(1024.0))",8170.731199875941
SFO,DEN,967,7897.166666666666,"List(1, 1, List(), List(967.0))",7716.353636628149
SFO,SEA,679,5624.383333333333,"List(1, 1, List(), List(679.0))",5420.551211797198
ORD,MIA,1197,9915.15,"List(1, 1, List(), List(1197.0))",9549.806962013976
SFO,IAD,2419,18747.25,"List(1, 1, List(), List(2419.0))",19291.024195150854
ORD,SEA,1721,13968.783333333335,"List(1, 1, List(), List(1721.0))",13726.891929414733
ATL,BOS,946,7315.733333333334,"List(1, 1, List(), List(946.0))",7548.951376484225
ATL,LAX,1947,15770.699999999999,"List(1, 1, List(), List(1947.0))",15528.459110011243


In [87]:
display(model, preppedDataDF, "fittedVsResiduals")

fitted values,residuals
10036.070670051087,48.89599661557986
10036.070670051087,174.69599661557913
19291.024195150854,383.50913818247864
17058.99405989854,88.65594010146015
17401.77011638372,-491.27011638371914
20821.559145038154,-412.2424783714887
8170.731199875941,294.33546679072697
9549.806962013976,185.79303798602268
5420.551211797198,-180.93454513053166
4838.629069392131,120.47093060786938


In [88]:
# Homework 10
# Twitter project

In [89]:
%sh
pip install requests requests_oauthlib

Requirement already satisfied: requests in /databricks/python3/lib/python3.7/site-packages (2.21.0)
Requirement already satisfied: requests_oauthlib in /databricks/python3/lib/python3.7/site-packages (1.3.0)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests) (2019.3.9)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests) (1.24.1)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests) (2.8)
Requirement already satisfied: oauthlib>=3.0.0 in /databricks/python3/lib/python3.7/site-packages (from requests_oauthlib) (3.1.0)
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [90]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [91]:
ACCESS_TOKEN = '273581743-siLMhZNxr3vx8w0XizHt9CMqW5ffsDHHuy8CJvMw'
ACCESS_SECRET = 'iiRX1RbdjksEWWK70iLmwzXMaZXEN8AsH4Z3ftjELryW1'
CONSUMER_KEY = 'lYjK0ZCqsPSW0QXliOBUSAR2Q'
CONSUMER_SECRET = 'dksTWPTrXajCkL7CwsNYNRi1DNdCt0OxSSI5seLyIsSjQD2KpM' 
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)

In [92]:
def get_tweets():
  url = 'https://stream.twitter.com/1.1/statuses/filter.json'
  query_data = [('language', 'en'), ('track', 'trump')]
  query_url = url + '?' + '&'.join(str(t[0]) + '='+ str(t[1]) for t in query_data)
  response = requests.get(query_url, auth = my_auth, stream = True)
  print(query_url, response)
  return response

In [93]:
def send_tweets_to_spark(http_resp, tcp_connection):
  for line in http_resp.iter_lines():
    try:
      full_tweet = json.loads(line)
      tweet_text = str(full_tweet['text'].encode('utf-8'))
      print('Tweet text: ' + tweet_text)
      print('-------------------')
      tcp_connection.send(tweet_text + '\n', 'utf-8')
    except:
      e = sys.exc_info()[0]
      print('Error #1: %s' % e)

In [94]:
TCP_IP = 'localhost'
TCP_PORT = 9012
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print('Waiting for TCP connection...')
conn, addr = s.accept()
print('Connected...Starting getting tweets.')
resp = get_tweets()
send_tweets_to_spark(resp, conn)

In [95]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SQLContext
import sys
import requests

In [96]:
def aggregate_tags_count(new_values, total_sum):
  return sum(new_values) + (total_sum or 0)

def get_sql_context_instance(spark_context):
  if ('sqlContextSingletonInstance' not in globals()):
    globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
  
def process_rdd(time, rdd):
  print('--------------%s-------------------' %str(time))
  try:
#     get spark sql singleton context from the current context
    sql_context = get_sql_context_instance(rdd.context)
  #   convert the RDD to Row RDD
    row_rdd = rdd.map(lambda w: Row(hashtag = w[0], hashtag_count = w[1]))
    hashtags_df = sql_context.createDataFrame(row_rdd)
    hashtags_df.registerTempTable('hashtags')
    hashtag_counts_df = sql_context.sql('select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10')
    hashtag_counts_df.show()
    send_df_to_dashboard(hashtag_counts_df)
  except:
    e = sys.exc_info()
    print('Error #2: %s' %e)
  
def send_df_to_dashboard(df):
  top_tags = [str(t.hashtag) for t in df.select('hashtag').collect()]
  tags_count = [p.hashtag_count for p in df.select('hashtag_count').collect()]
  url = 'http://localhost:9009/updateData'
  request_data = {'label': str(top_tags), 'data': str(tags_count)}
  response = requests.post(url, data = request_data)
    

In [97]:
conf = SparkConf()
conf.setAppName('TwitterStreamApp')
# sc = SparkContext(conf = conf)
sc.setLogLevel('ERROR')

ssc = StreamingContext(sc, 2)
ssc.checkpoint('/FileStore/checkpoint_TwitterApp')
dataStream = ssc.socketTextStream('localhost', 9009)

In [98]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag, 1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# addint the count of each hashtag to its last count
tags_totals = hashtags.reduceByKey(lambda a,b: a+b)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation

# You need to open this up the first time, and comment it out in subsequent runnings
# ssc.start()


# wait for the streaming to finish
ssc.awaitTermination()

In [99]:
import socket
import sys
import requests
import requests_oauthlib
import json

# Replace the values below with yours
# ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN'
# ACCESS_SECRET = 'YOUR_ACCESS_SECRET'
# CONSUMER_KEY = 'YOUR_CONSUMER_KEY'
# CONSUMER_SECRET = 'YOUR_CONSUMER_SECRET'
# my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)


def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text'].encode("utf-8") + '\n' # pyspark can't accept stream, add '\n'
            print("Tweet Text: " + tweet_text)
            print ("------------------------------------------")
            tcp_connection.send(tweet_text + '\n')
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)


def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('track','umbc')]
    #query_data = [(locations', '-122.75,36.8,-121.75,37.8,-74,40,-73,41'), ('track', '#')] #this location value is San Francisco & NYC
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response


TCP_IP = "localhost"
TCP_PORT = 9010
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp,conn)


